In [1]:
import pandas as pd
import data_fetching as df
from datetime import datetime
from queries import order_query
from queries import product_query
import os
import shopify

In [2]:
# Variables for store
shop_url = os.environ.get("SHOP_URL", "")
api_version = "2024-01"
private_app_password = os.environ.get("PRIVATE_APP_PASSWORD", "")

In [3]:
api_session = shopify.Session(shop_url, api_version, private_app_password)
shopify.ShopifyResource.activate_session(api_session)
client = shopify.GraphQL()

In [4]:
bulk_order_query = df.bulk_operation(order_query)
bulk_orders = shopify.GraphQL().execute(bulk_order_query)
bulk_orders

'{"data":{"bulkOperationRunQuery":{"bulkOperation":{"id":"gid://shopify/BulkOperation/4480823984468","status":"CREATED"},"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}'

In [5]:
order_status = df.bulk_status(shopify.GraphQL(), bulk_orders)
order_status

{'data': {'node': {'id': 'gid://shopify/BulkOperation/4480823984468',
   'status': 'COMPLETED',
   'errorCode': None,
   'createdAt': '2024-01-31T11:59:52Z',
   'completedAt': '2024-01-31T11:59:55Z',
   'objectCount': '5',
   'url': 'https://storage.googleapis.com/shopify-tiers-assets-prod-us-east1/gi8vcv1qnf2ryy22vrpks9l17pha?GoogleAccessId=assets-us-prod%40shopify-tiers.iam.gserviceaccount.com&Expires=1707307195&Signature=A6YIdhceyvZEmYtVtY1TnVchJR5okQHuutHpeGgklzUf9T6hVBtvcQxmZrwZo4VSYgH2RLiVK7Dx4u7lQwdzrkzz0ynesUMrUQ%2B9U%2Bz2yn1X0tvgIb%2FbK1XXwd%2FoHwGS%2BUIiM3ENBLJcPh2hTB3PnKau%2BTZNYcm0z5r%2FpyMZBw3AStLDRsqvcpZNi8ODXWM%2FCYyV5GS9o7MU%2F1YEhFERbHecWYgGrHlFzOYM%2FcFOSzRZ9R2l9jnjOPQGYkoXYmzmevV4Zd0ReqeehuhyS9pLOzj67J9w57dJHGgGI%2BXA1dapgQDjRHN7T22hXf0Z1O63LWrHEVO8fcWmD%2FtLAI7OHg%3D%3D&response-content-disposition=attachment%3B+filename%3D%22bulk-4480823984468.jsonl%22%3B+filename%2A%3DUTF-8%27%27bulk-4480823984468.jsonl&response-content-type=application%2Fjsonl',
   'partialDataUr

In [6]:
orders = df.get_bulk_data(order_status)
orders

[{'id': 'gid://shopify/Order/5940801995092',
  'name': '#1001',
  'displayFinancialStatus': 'PAID',
  'displayFulfillmentStatus': 'UNFULFILLED',
  'customerAcceptsMarketing': False,
  'currencyCode': 'EUR',
  'subtotalPrice': '1099.00',
  'discountCode': None,
  'note': None,
  'phone': None,
  'subtotalLineItemsQuantity': 1,
  'tags': [],
  'customer': None,
  'app': {'id': 'gid://shopify/App/1354745', 'name': 'Draft Orders'},
  'totalTax': '0.00',
  'totalPrice': '1099.00',
  'createdAt': '2024-01-28T14:12:10Z',
  'totalDiscountsSet': {'shopMoney': {'amount': '0.0'}},
  'shippingLine': None,
  'billingAddress': None,
  'channelInformation': None,
  'shippingAddress': None},
 {'id': 'gid://shopify/Order/5942622257492',
  'name': '#1002',
  'displayFinancialStatus': 'PAID',
  'displayFulfillmentStatus': 'UNFULFILLED',
  'customerAcceptsMarketing': False,
  'currencyCode': 'EUR',
  'subtotalPrice': '1099.00',
  'discountCode': None,
  'note': None,
  'phone': None,
  'subtotalLineItemsQ

In [7]:
bulk_product_query = df.bulk_operation(product_query)
bulk_products = shopify.GraphQL().execute(bulk_product_query)
bulk_products

'{"data":{"bulkOperationRunQuery":{"bulkOperation":{"id":"gid://shopify/BulkOperation/4480824115540","status":"CREATED"},"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":2000.0,"currentlyAvailable":1990,"restoreRate":100.0}}}}'

In [8]:
product_status = df.bulk_status(shopify.GraphQL(), bulk_products)
product_status

{'data': {'node': {'id': 'gid://shopify/BulkOperation/4480824115540',
   'status': 'COMPLETED',
   'errorCode': None,
   'createdAt': '2024-01-31T11:59:56Z',
   'completedAt': '2024-01-31T11:59:59Z',
   'objectCount': '14',
   'url': 'https://storage.googleapis.com/shopify-tiers-assets-prod-us-east1/odu9u2nknjyo6jv8lk7ai123elwl?GoogleAccessId=assets-us-prod%40shopify-tiers.iam.gserviceaccount.com&Expires=1707307199&Signature=Uh7p4%2BwFu8XmWsX2ollyZDHGXbmL8wk3%2FfA6kaQYQSdbZHc3%2FpEaOfGiwCZ01eP6kiGjDcVN3gU3isGbsUL%2F6K3ww27UCQcUZpd3kv%2F4sehi7%2FRIzlgvUbNmTASmTxbV1BPlQo20U8bmpSr%2BKgmKpa7vH6JhDacQYbIpkv98ULj9Yi0mjfg051kLThNgN3k9gLHZWgYiNfaQv%2BeuJs2oCrFj0k6VS271Td57wkZFOsNOVJQvTFOWBLABduITSeD2HpEEj64WYLk0idtF%2FqC1uf%2BJJ0%2Fzu7iEdbuz2bOrbMcp9Kh%2B%2Fb1q21UuGSsl0SIMgmAB1y%2FxUDp06lMeDeYcbg%3D%3D&response-content-disposition=attachment%3B+filename%3D%22bulk-4480824115540.jsonl%22%3B+filename%2A%3DUTF-8%27%27bulk-4480824115540.jsonl&response-content-type=application%2Fjsonl',
   'partialD

In [9]:
products = df.get_bulk_data(product_status)
products

[{'id': 'gid://shopify/InventoryItem/49513063907668',
  'sku': '',
  'variant': {'id': 'gid://shopify/ProductVariant/47416123687252',
   'displayName': 'Puff - Silver',
   'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/Palocerasmetahumanpuff.jpg?v=1704370914'},
   'inventoryQuantity': -3,
   'inventoryItem': {'id': 'gid://shopify/InventoryItem/49513063907668'}}},
 {'id': 'gid://shopify/InventoryLevel/124769108308?inventory_item_id=49513063907668',
  'quantities': [{'name': 'available', 'quantity': -3},
   {'name': 'on_hand', 'quantity': 0},
   {'name': 'committed', 'quantity': 3},
   {'name': 'quality_control', 'quantity': 0}],
  '__parentId': 'gid://shopify/InventoryItem/49513063907668'},
 {'id': 'gid://shopify/InventoryItem/49513063940436',
  'sku': '',
  'variant': {'id': 'gid://shopify/ProductVariant/47416123720020',
   'displayName': 'Puff - Gold',
   'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/18.jpg?v=1704371756'},
   'inven

In [16]:
# Dictionary to store inventory items by ID for easy lookup
inventory_items = {}

# Index inventory items by ID for quick lookup
# Index inventory items by ID for quick lookup
for item in products:
    if 'variant' in item and 'inventoryItem' in item['variant']:
        inventory_id = item['variant']['inventoryItem']['id']
        inventory_items[inventory_id] = item['variant']

# Update missing parts of the data
for item in products:
    if '__parentId' in item:
        parent_id = item['__parentId']
        if parent_id in inventory_items:
            parent = inventory_items[parent_id]
            for key in ['displayName', 'image', 'sku', 'inventoryItem']:
                if key in parent:
                    item[key] = parent[key]

for product in products:
    print(item)

{'id': 'gid://shopify/InventoryItem/49513063907668', 'sku': '', 'variant': {'id': 'gid://shopify/ProductVariant/47416123687252', 'displayName': 'Puff - Silver', 'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/Palocerasmetahumanpuff.jpg?v=1704370914'}, 'inventoryQuantity': -3, 'inventoryItem': {'id': 'gid://shopify/InventoryItem/49513063907668'}}}
{'id': 'gid://shopify/InventoryLevel/124769108308?inventory_item_id=49513063907668', 'quantities': [{'name': 'available', 'quantity': -3}, {'name': 'on_hand', 'quantity': 0}, {'name': 'committed', 'quantity': 3}, {'name': 'quality_control', 'quantity': 0}], '__parentId': 'gid://shopify/InventoryItem/49513063907668', 'displayName': 'Puff - Silver', 'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/Palocerasmetahumanpuff.jpg?v=1704370914'}, 'inventoryItem': {'id': 'gid://shopify/InventoryItem/49513063907668'}}
{'id': 'gid://shopify/InventoryItem/49513063940436', 'sku': '', 'variant': {'id': 'gid://

In [11]:
ids = []
skus = []
displayNames = []
imageUrls = []
inventoryQuantities = []
product_result = []
for product in products:
    if 'quantities' in product:
        product_result.append(product)

product_result


[{'id': 'gid://shopify/InventoryLevel/124769108308?inventory_item_id=49513063907668',
  'quantities': [{'name': 'available', 'quantity': -3},
   {'name': 'on_hand', 'quantity': 0},
   {'name': 'committed', 'quantity': 3},
   {'name': 'quality_control', 'quantity': 0}],
  '__parentId': 'gid://shopify/InventoryItem/49513063907668',
  'displayName': 'Puff - Silver',
  'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/Palocerasmetahumanpuff.jpg?v=1704370914'},
  'inventoryItem': {'id': 'gid://shopify/InventoryItem/49513063907668'}},
 {'id': 'gid://shopify/InventoryLevel/124769108308?inventory_item_id=49513063940436',
  'quantities': [{'name': 'available', 'quantity': -2},
   {'name': 'on_hand', 'quantity': -1},
   {'name': 'committed', 'quantity': 1},
   {'name': 'quality_control', 'quantity': 0}],
  '__parentId': 'gid://shopify/InventoryItem/49513063940436',
  'displayName': 'Puff - Gold',
  'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/18

In [19]:
# Create lists to store each column of the data
ids = []
parent_ids = []
display_names = []
image_urls = []
available_quantities = []
on_hand_quantities = []
committed_quantities = []
skus = []
unavailable_quantities = []
# Extract data from the provided structure
for product in product_result:

    display_names.append(product['displayName'])
    quantities = product['quantities']
    available_quantities.append(next((q['quantity'] for q in quantities if q['name'] == 'available'), ''))
    on_hand_quantities.append(next((q['quantity'] for q in quantities if q['name'] == 'on_hand'), ''))
    committed_quantities.append(next((q['quantity'] for q in quantities if q['name'] == 'committed'), ''))
    unavailable_quantities.append(next((q['quantity'] for q in quantities if q['name'] == 'quality_control'), ''))

# Create a DataFrame
product_df = pd.DataFrame({
    'Display Name': display_names,
    'Unavailable': unavailable_quantities,
    'Available Quantity': available_quantities,
    'On Hand Quantity': on_hand_quantities,
    'Committed Quantity': committed_quantities
})
product_df

,Display Name,Unavailable,Available Quantity,On Hand Quantity,Committed Quantity
0,Puff - Silver,0,-3,0,3
1,Puff - Gold,0,-2,-1,1
2,Pebble Teal X9 sunglasses - Default Title,0,10,10,0
3,Pebble Teal X8 sunglasses - Default Title,0,10,10,0
4,Pebble Teal X7 sunglasses - Default Title,0,10,10,0
5,Pebble Teal X6 sunglasses - Default Title,0,10,10,0
6,Pebble Teal X5 sunglasses - Default Title,0,10,10,0


{'id': 'gid://shopify/InventoryLevel/124769108308?inventory_item_id=49513063907668',
 'quantities': [{'name': 'available', 'quantity': -3},
  {'name': 'on_hand', 'quantity': 0},
  {'name': 'committed', 'quantity': 3},
  {'name': 'quality_control', 'quantity': 0}],
 '__parentId': 'gid://shopify/InventoryItem/49513063907668',
 'displayName': 'Puff - Silver',
 'image': {'url': 'https://cdn.shopify.com/s/files/1/0797/5328/1876/files/Palocerasmetahumanpuff.jpg?v=1704370914'},
 'inventoryItem': {'id': 'gid://shopify/InventoryItem/49513063907668'}}

In [13]:
from data_validation import order_data_checking, line_item_data_checking, shipping_line_data_checking, \
                            fulfillment_data_checking, customer_data_checking, note_attributes_data_checking, \
                            tax_line_data_checking, assign_value
data = []
for order in orders:
    order_name, order_email, order_financial_status, order_fulfillment_status, order_buyer_accepts_marketing, \
    order_currency, order_subtotal_price, order_total_tax, order_total_price, order_total_discounts,  \
    order_created_at, order_note, order_cancelled_at, order_payment_method, order_refunds, order_payment_reference, \
    order_id, order_tags, order_source, order_phone, order_duties, order_payment_terms_name, order_discount_code, \
    order_subtotal_line_items_quantity = order_data_checking(order)
    
    discount_code = ""
    
    shipping_price, shipping_method = "", ""
    
    billing_first_name, billing_last_name, billing_address1, billing_address2, billing_company, \
    billing_city, billing_zip, billing_province, billing_country, billing_phone, billing_province_name \
    = "", "", "", "", "", "" "", "", "", "", "", ""

    shipping_first_name, shipping_last_name, shipping_address1, shipping_address2, shipping_company, \
    shipping_city, shipping_zip, shipping_province, shipping_country, shipping_phone, shipping_province_name \
    = "", "", "", "", "", "", "", "", "", "", ""
    
    note_attributes = ""
    
    fulfillment_at = ""

    line_item_quantity, line_item_name, line_item_price, line_item_sku, \
    line_item_requires_shipping, line_item_taxable, line_item_fulfillment_status, \
    line_item_vendor, line_item_discount = "", "", "", "", "", "", "", "", ""
    
    tax_line_name, tax_line_rate, tax_line_value = "", "", ""
    
    order_data = {
        "Order": order_name["name"] if order_name != "" else "",
        "Created At": datetime.fromisoformat(order_created_at["createdAt"].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S') if order_created_at != "" else "",
        "Customer": order["customer"]["displayName"] if order["customer"] != None else "",
        "Channel": order["channelInformation"]["displayName"] if order["channelInformation"] != None else "",
        "Total": order_total_price["totalPrice"] if order_total_price != None else "",
        "Financial Status": order_financial_status["displayFinancialStatus"].lower() if order_financial_status != None else "",
        "Fulfillment Status": order_fulfillment_status["displayFulfillmentStatus"].lower() if order_fulfillment_status != None else "",
        "Items": order_subtotal_line_items_quantity["subtotalLineItemsQuantity"] if order_subtotal_line_items_quantity != None else "",
        "Tags": ', '.join(order_tags["tags"]) if order_tags != [] else "",
        "Delivery Status": "",
        "Delivery Method": order["shippingLine"]["title"] if order["shippingLine"] != None else "",
    }
        
    data.append(order_data)

order_df  = pd.DataFrame(data)
order_df.fillna("", inplace=True)
order_df

,Order,Created At,Customer,Channel,Total,Financial Status,Fulfillment Status,Items,Tags,Delivery Status,Delivery Method
0,#1001,2024-01-28 14:12:10,,,1099.00,paid,unfulfilled,1,,,
1,#1002,2024-01-30 02:12:28,Ryan Nguyen,,1099.00,paid,unfulfilled,1,,,
2,#1003,2024-01-30 03:03:02,,,1099.00,paid,unfulfilled,1,Hello,,
3,#1004,2024-01-30 03:06:46,,,1099.00,paid,unfulfilled,1,"Hello, Hi",,
4,#1005,2024-01-30 03:14:07,Ryan Nguyen,,1009.10,pending,fulfilled,1,Gold,,Free Shipping


In [14]:
product_status = df.bulk_status(client, bulk_products)
product_status

AttributeError: 'DataFrame' object has no attribute 'bulk_status'